## PySpark 설치

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=f4eb7ef680f8e10c5f969e82ac99bd953a7e942d9a0f6daa8ac20114fd42c7c1
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
!cd /usr/local/lib/python3.10/dist-packages/pyspark/jars && wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.30/redshift-jdbc42-2.1.0.30.jar

--2023-01-23 19:38:57--  https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.30/redshift-jdbc42-2.1.0.30.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.100.46, 52.217.173.72, 52.217.224.184, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.100.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2413910 (2.3M) [application/java-archive]
Saving to: ‘redshift-jdbc42-2.1.0.30.jar’

RedshiftJDBC42-no-a 100%[===================>]   2.30M  9.14MB/s    in 0.3s    

2023-01-23 19:38:57 (9.14 MB/s) - ‘redshift-jdbc42-2.1.0.30.jar’ saved [2413910/2413910]



In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL #1") \
    .config("spark.jars", "/usr/local/lib/python3.10/dist-packages/pyspark/jars/redshift-jdbc42-2.1.0.30.jar") \
    .getOrCreate()

## Redshift 상의 다음 테이블을 이용하여 월별 채널별 매출과 방문자 정보 계산하기
user_session_channel, session_timestamp, session_transaction

In [4]:
# Redshift와 연결해서 DataFrame으로 로딩하기
url = "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234"

df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

df_session_transaction = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_transaction") \
    .load()

In [5]:
df_user_session_channel.createOrReplaceTempView("user_session_channel")
df_session_timestamp.createOrReplaceTempView("session_timestamp")
df_session_transaction.createOrReplaceTempView("session_transaction")

In [6]:
spark.table("session_timestamp").show(5)

+--------------------+--------------------+
|           sessionid|                  ts|
+--------------------+--------------------+
|00029153d12ae1c9a...|2019-10-18 14:14:...|
|0004289ee1c7b8b08...|2019-11-16 21:20:...|
|0006246bee639c7a7...|2019-08-10 16:33:...|
|0006dd05ea1e999dd...|2019-07-06 19:54:...|
|000958fdaefe0dd06...|2019-11-02 14:52:...|
+--------------------+--------------------+
only showing top 5 rows



In [7]:
df_user_session_channel.show(5)

+------+--------------------+---------+
|userid|           sessionid|  channel|
+------+--------------------+---------+
|  1651|0004289ee1c7b8b08...|  Organic|
|  1197|00053f5e11d1fe4e4...| Facebook|
|  1401|00056c20eb5a02958...| Facebook|
|  1399|00063cb5da1826feb...| Facebook|
|  1667|000958fdaefe0dd06...|Instagram|
+------+--------------------+---------+
only showing top 5 rows



In [8]:
df_session_timestamp.show(5)

+--------------------+--------------------+
|           sessionid|                  ts|
+--------------------+--------------------+
|00029153d12ae1c9a...|2019-10-18 14:14:...|
|0004289ee1c7b8b08...|2019-11-16 21:20:...|
|0006246bee639c7a7...|2019-08-10 16:33:...|
|0006dd05ea1e999dd...|2019-07-06 19:54:...|
|000958fdaefe0dd06...|2019-11-02 14:52:...|
+--------------------+--------------------+
only showing top 5 rows



In [9]:
df_session_transaction.show(5)

+--------------------+--------+------+
|           sessionid|refunded|amount|
+--------------------+--------+------+
|00029153d12ae1c9a...|   false|    85|
|008909bd27b680698...|   false|    13|
|0107acb41ef20db22...|   false|    16|
|018544a2c48077d2c...|   false|    39|
|020c38173caff0203...|   false|    61|
+--------------------+--------+------+
only showing top 5 rows



## 월별 채널별 총 방문자와 매출 발생 방문자 계산

In [10]:
# JOIN key가 정말 하나씩만 존재하나?
spark.sql("""SELECT sessionid, COUNT(1) count
FROM user_session_channel
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1""").show() 

+--------------------+-----+
|           sessionid|count|
+--------------------+-----+
|0226aa5193c66d990...|    1|
+--------------------+-----+



In [11]:
# JOIN key가 정말 하나씩만 존재하나?
spark.sql("""SELECT sessionid, COUNT(1) count
FROM session_transaction
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1""").show() 

+--------------------+-----+
|           sessionid|count|
+--------------------+-----+
|532ff98823e7d1433...|    1|
+--------------------+-----+



In [12]:
# JOIN key가 정말 하나씩만 존재하나?
spark.sql("""SELECT sessionid, COUNT(1) count
FROM session_timestamp
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1""").show() 

+--------------------+-----+
|           sessionid|count|
+--------------------+-----+
|0135456d6a3c1051f...|    1|
+--------------------+-----+



 ### 월별 채널별 총 방문자 계산

In [13]:
mon_channel_rev_df = spark.sql("""
    SELECT LEFT(sti.ts, 7) year_month,
      usc.channel channel,
      COUNT(DISTINCT userid) total_visitors
    FROM user_session_channel usc
    LEFT JOIN session_timestamp sti ON usc.sessionid = sti.sessionid
    GROUP BY 1 ,2
    ORDER BY 1, 2""")

In [14]:
mon_channel_rev_df.show()

+----------+---------+--------------+
|year_month|  channel|total_visitors|
+----------+---------+--------------+
|   2019-05| Facebook|           247|
|   2019-05|   Google|           253|
|   2019-05|Instagram|           234|
|   2019-05|    Naver|           237|
|   2019-05|  Organic|           238|
|   2019-05|  Youtube|           244|
|   2019-06| Facebook|           414|
|   2019-06|   Google|           412|
|   2019-06|Instagram|           410|
|   2019-06|    Naver|           398|
|   2019-06|  Organic|           416|
|   2019-06|  Youtube|           400|
|   2019-07| Facebook|           558|
|   2019-07|   Google|           556|
|   2019-07|Instagram|           567|
|   2019-07|    Naver|           553|
|   2019-07|  Organic|           557|
|   2019-07|  Youtube|           564|
|   2019-08| Facebook|           611|
|   2019-08|   Google|           610|
+----------+---------+--------------+
only showing top 20 rows



 ### 월별 채널별 총 방문자와 구매 방문자 계산

In [21]:
mon_channel_rev_df = spark.sql("""
    SELECT LEFT(sti.ts, 7) year_month,
      usc.channel channel,
      COUNT(DISTINCT userid) total_visitors,
      COUNT(DISTINCT CASE WHEN amount is not NULL THEN userid END) paid_visitors
    FROM user_session_channel usc
    LEFT JOIN session_timestamp sti ON usc.sessionid = sti.sessionid
    LEFT JOIN session_transaction str ON usc.sessionid = str.sessionid
    GROUP BY 1 ,2
    ORDER BY 1, 2""")

In [22]:
mon_channel_rev_df.show()

+----------+---------+--------------+-------------+
|year_month|  channel|total_visitors|paid_visitors|
+----------+---------+--------------+-------------+
|   2019-05| Facebook|           247|           14|
|   2019-05|   Google|           253|           10|
|   2019-05|Instagram|           234|           11|
|   2019-05|    Naver|           237|           11|
|   2019-05|  Organic|           238|           17|
|   2019-05|  Youtube|           244|           10|
|   2019-06| Facebook|           414|           22|
|   2019-06|   Google|           412|           13|
|   2019-06|Instagram|           410|           21|
|   2019-06|    Naver|           398|           15|
|   2019-06|  Organic|           416|           14|
|   2019-06|  Youtube|           400|           17|
|   2019-07| Facebook|           558|           32|
|   2019-07|   Google|           556|           21|
|   2019-07|Instagram|           567|           25|
|   2019-07|    Naver|           553|           19|
|   2019-07|

## 월별 채널별 총 매출액 (리펀드 포함), 총 방문자, 매출 발생 방문자, 전환률 계산

In [29]:
 mon_channel_rev_df = spark.sql("""
  SELECT LEFT(ts, 7) month,
       usc.channel,
       COUNT(DISTINCT userid) uniqueUsers,
       COUNT(DISTINCT (CASE WHEN amount >= 0 THEN userid END)) paidUsers,
       SUM(amount) grossRevenue,
       SUM(CASE WHEN refunded is not True THEN amount END) netRevenue,
       ROUND(COUNT(DISTINCT CASE WHEN amount >= 0 THEN userid END)*100
          / COUNT(DISTINCT userid), 2) conversionRate
   FROM user_session_channel usc
   LEFT JOIN session_timestamp t ON t.sessionid = usc.sessionid
   LEFT JOIN session_transaction st ON st.sessionid = usc.sessionid
   GROUP BY 1, 2
   ORDER BY 1, 2;
""")

In [30]:
mon_channel_rev_df.show()

+-------+---------+-----------+---------+------------+----------+--------------+
|  month|  channel|uniqueUsers|paidUsers|grossRevenue|netRevenue|conversionRate|
+-------+---------+-----------+---------+------------+----------+--------------+
|2019-05| Facebook|        247|       14|        1199|       997|          5.67|
|2019-05|   Google|        253|       10|         580|       580|          3.95|
|2019-05|Instagram|        234|       11|         959|       770|           4.7|
|2019-05|    Naver|        237|       11|         867|       844|          4.64|
|2019-05|  Organic|        238|       17|        1846|      1571|          7.14|
|2019-05|  Youtube|        244|       10|         529|       529|           4.1|
|2019-06| Facebook|        414|       22|        1578|      1578|          5.31|
|2019-06|   Google|        412|       13|         947|       947|          3.16|
|2019-06|Instagram|        410|       21|        1462|      1418|          5.12|
|2019-06|    Naver|        3